In [ ]:
from datasets import load_dataset
from utils import jpeg_compress, webp_compress, pad, crop, nn_compress, hific_lo_compress
import compressai
import evaluate
from transformers import AutoFeatureExtractor, SegformerForSemanticSegmentation
from piq import LPIPS
from torch.nn import MSELoss

In [ ]:
def jpeg_compress_ade(sample):
    img,bpp = jpeg_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample
    
def webp_compress_ade(sample):
    img,bpp = webp_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample

net_mbt2018 = compressai.zoo.mbt2018(quality=1, pretrained=True).eval().to("cpu")
def mbt2018_compress_ade(sample):
    img,bpp = nn_compress(sample['image'],net_mbt2018,"cpu")
    sample['image'] = img
    sample['bpp'] = bpp
    return sample

def hific_lo_compress_ade(sample):
    img,bpp = hific_lo_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample

In [ ]:
image_compression_methods = [
    jpeg_compress_ade,
    webp_compress_ade,
    mbt2018_compress_ade,
    hific_lo_compress_ade
]

In [ ]:
ade20k = [load_dataset("scene_parse_150",split='validation')]

In [ ]:
for method in image_compression_methods:
    ade20k.append(ade20k[0].map(method))

In [ ]:
image_bpp = [np.mean(method['bpp']) for method in ade20k[1:]]
image_bpp

In [ ]:
lpips_metric = LPIPS()
mse_metric = MSELoss()

In [ ]:
%%time
lpips = []
mse = []
for method in ade20k:
    method = method.with_format("torch")
    lpips.append([])
    mse.append([])
    for i_sample,sample in enumerate(beans[0].with_format("torch")):
        compressed_sample = method[i_sample]
        reference = sample['image'].to(torch.float32).permute(2,0,1).unsqueeze(0)
        distorted = compressed_sample['image'].to(torch.float32).permute(2,0,1).unsqueeze(0)
        lpips[-1].append(lpips_metric(reference,distorted).detach().item())
        mse[-1].append(mse_metric(reference,distorted).detach().item())

In [ ]:
PSNR = [[20*np.log10(255)-10*np.log10(d) for d in m] for m in mse]
PSNR = [np.mean(d) for d in PSNR]
PSNR

In [ ]:
neg_log_lpips = [[-10*np.log10(d) for d in m] for m in lpips]
neg_log_lpips = [np.mean(d) for d in neg_log_lpips]
neg_log_lpips